<a href="https://colab.research.google.com/github/penguin1109/MOAI/blob/main/Unetpp_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 매번 새롭게 trainset과 valid set을 나누어 주었었기 때문에 unbalanced class문제를 해결하기 위해서 적용해야 하는 loss의 weigth를 매번 새롭게 계산했다.
- classification을 해 주기 위해서 마지막 출력층에 sigmoid를 사용하였다.
- 각 pixel이 유일한 class로 각각 분류되어야 하는 상황이기 때문에 이와 같은 경우에는 softmax보다 loss값에 처벌을 더 강하게 하는 log softmax를 적용하는 것이 낫다.

In [ ]:
!pip install pydicom
!pip install albumentations==0.4.6

     |████████████████████████████████| 2.0 MB 7.8 MB/s 
     |████████████████████████████████| 117 kB 7.8 MB/s 
     |████████████████████████████████| 948 kB 57.9 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65172 sha256=0bfd0b267c08ada355cab2bd6328fdc530c2ab6295a0496bd408e9de2baa3913
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
# Hyperparameters
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 3e-4
BATCH_SIZE = 4
NUM_EPOCHS = 17
IMAGE_WIDTH = 512
IMAGE_HEIGHT = 512
LOAD_MODEL = True
PIN_MEMORY = True
NUM_WORKERS = 1
TRAIN_IMAGE_DIR = '/content/drive/Shareddrives/Kaggle_MOAI2021/data/train/DICOM'
TRAIN_MASK_DIR = '/content/drive/Shareddrives/Kaggle_MOAI2021/data/train/Label'
TEST_IMAGE_DIR = '/content/drive/Shareddrives/Kaggle_MOAI2021/data/test'

SIGMOID = False
CHECKPOINT_DIR = '/content/drive/Shareddrives/Kaggle_MOAI2021/checkpoints/unetppFin_checkpoint.pth.tar'
FILENAME = 'unetppFin_checkpoint.pth.tar'



In [ ]:
import cv2
from glob import glob
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
from pydicom import dcmread
import matplotlib.image as mpimg

def get_mask_bool(mask, type = None):
    mask_1 = mask < 0.00784314
    mask_2 = mask >= 0.00392157
    mask_3 = np.logical_and(mask_1, mask_2)
    if type == 'tumor':
        mask = (mask >= 0.00784314).astype(float)
    elif type == 'organ':
        mask = mask_3.astype(float)
    elif type == "all":
        mask = (mask!=0).astype(float)
    else:
        mask = (mask < 0.00392157).astype(float)
    if 1.0 not in mask:
      return False
    else:
      return True


In [ ]:
def list_with(type, image_dir, mask_dir):
  dcom_dirs = np.array(sorted(glob(image_dir + '/*/*')))
  mask_dirs = np.array(sorted(glob(mask_dir + '/*/*')))
  imgs, masks = [],[]
  if type == 'tumor':
    for idx, dir in enumerate(mask_dirs):
      mask = mpimg.imread(dir)
      mask = np.around(mask, 8)
      if get_mask_bool(mask, 'tumor'):
        imgs.append(dcom_dirs[idx])
        masks.append(mask_dirs[idx])
  elif type == 'organ':
    for idx, dir in enumerate(mask_dirs):
      mask = mpimg.imread(dir)
      mask = np.around(mask, 8)
      if get_mask_bool(mask, 'organ'):
        imgs.append(dcom_dirs[idx])
        masks.append(mask_dirs[idx])
  else:
    for idx, dir in enumerate(mask_dirs):
      mask = mpimg.imread(dir)
      mask = np.around(mask, 8)
      if get_mask_bool(mask, 'all'):
        imgs.append(dcom_dirs[idx])
        masks.append(mask_dirs[idx])

  return imgs, masks



In [ ]:
#dcom_dirs, mask_dirs = list_with('all', TRAIN_IMAGE_DIR, TRAIN_MASK_DIR)
#dcom_dirs, mask_dirs = np.array(dcom_dirs), np.array(mask_dirs)
#print(len(dcom_dirs), len(mask_dirs))

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import pydicom
from sklearn.metrics import jaccard_similarity_score

def transform_to_hu(medical_image, image):
    hu_image = image * medical_image.RescaleSlope + medical_image.RescaleIntercept
    hu_image[hu_image < -1024] = -1024
    return hu_image

def window_image(image, window_center, window_width):
    window_image = image.copy()
    image_min = window_center - (window_width / 2)
    image_max = window_center + (window_width / 2)
    window_image[window_image < image_min] = image_min
    window_image[window_image > image_max] = image_max
    return window_image

def resize_normalize(image):
    image = np.array(image, dtype=np.float64)
    image -= np.min(image)
    image /= np.max(image)
    return image

def read_dicom(path, window_widht, window_level, channel):
    image_medical = pydicom.dcmread(path)
    image_data = image_medical.pixel_array

    image_hu = transform_to_hu(image_medical, image_data)
    image_window = window_image(image_hu.copy(), window_level, window_widht)
    image_window_norm = resize_normalize(image_window)


    image_window_norm = np.expand_dims(image_window_norm, axis=-1)   # (512, 512, 1)
    if channel == 3:
      image_ths = np.concatenate([image_window_norm, image_window_norm, image_window_norm], axis=2)   # (512, 512, 3)
    else:
      image_ths = image_window_norm
    return image_ths

In [ ]:
import torch
import argparse
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

def str2bool(v):
    if v.lower() in ['true', 1]:
        return True
    elif v.lower() in ['false', 0]:
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def save_checkpoint(state, filename = FILENAME):
    root = '/content/drive/Shareddrives/Kaggle_MOAI2021/checkpoints'
    print("=>saving checkpoint...")
    torch.save(state, root + '/' + filename)

def load_checkpoint(checkpoint, model):
    print("=>loading checkpoint...")
    model.load_state_dict(checkpoint['state_dict'])

def get_mask(mask, type = None):
    mask_1 = mask < 0.00784314
    mask_2 = mask >= 0.00392157
    mask_3 = np.logical_and(mask_1, mask_2)
    if type == 'tumor':
        mask = (mask >= 0.00784314).astype(float)
    elif type == 'organ':
        mask = mask_3.astype(float)
    elif type == "all":
        mask = (mask!=0).astype(float)
    else:
        mask = (mask < 0.00392157).astype(float)
    return mask

def get_loaders(train_dir,valid_dir, batch_size, train_transform, valid_transform, num_workers = 1, pin_memory = True):
    # 매번 validataion dataset을 새롭게 설정해 준다고 하면 (전체 dataset에서 비율을 정해서 해줌)
    train_ds = MOAIDataset(
        train_dir,
        transform = train_transform
    )
    
    train_loader = DataLoader(
        train_ds,
        batch_size = batch_size, shuffle = True,
        num_workers = num_workers, pin_memory=pin_memory
    )

    valid_ds = MOAIDataset(
        valid_dir,
        transform = valid_transform
    )

    valid_loader = DataLoader(
        valid_ds, 
        batch_size = batch_size, shuffle = True,
        num_workers = num_workers, pin_memory = pin_memory
    )

    return train_loader, valid_loader

def check_accuracy(loader, scheduler, model, device = "cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()  # unet모델의 가중치를 학습하는게 아니라 evaluation 단계에 들어갔음을 의미하는 명령어

    with torch.no_grad():
        for image, mask in loader:
            image = image.to(device)
            mask = mask.permute(0, 3, 1, 2).to(device)
            print('making predictions...')
            if SIGMOID:
              predictions = model(image.float())
            else:
              predictions = torch.sigmoid(model(image.float()))
            print("prediction made...")

            result = (predictions > 0.5)[0].float().cpu().permute(1, 2, 0).detach().numpy()
            visualize_mask(to_cpu(predictions)[0].permute(1, 2, 0).numpy())
            visualize_mask(result)

            predictions = (predictions > 0.5).float()
            num_correct += (predictions == mask).sum()
            num_pixels += torch.numel(predictions)
            dice_score += (2 * (predictions * mask).sum()) / ((predictions + mask).sum() + 1e-9)
            
    scheduler.step(dice_score/len(loader))
        
    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.3f}")
    print(f"Got DICE Score : {dice_score/len(loader)}")
    model.train()

def visualize(inputs, outputs, masks, channel):
  if not SIGMOID:
    outputs = torch.sigmoid(outputs)
    
  if torch.is_tensor(inputs):
    inputs = inputs.data.detach().cpu()

  if torch.is_tensor(outputs):
    outputs = (outputs).data.detach().cpu()
    outputs = (outputs>0.5).float()
    
  if torch.is_tensor(masks):
    masks = masks.data.detach().cpu()

  if channel == 1:
    for i in range(BATCH_SIZE):
      imgs = [inputs[i,:,:,0], outputs[i,0,:,:], masks[i,:,:,0]]
      plt.figure(figsize = (6, 6))
      for j in range(len(imgs)):
        plt.subplot(1, len(imgs), j+1)
        plt.imshow(imgs[j])
      plt.show()
  elif channel == 2:
    for i in range(BATCH_SIZE):
      imgs = [inputs[i,0,:,:], outputs[i].permute(1, 2, 0)[:,:,0],masks[i,:,:,0],outputs[i].permute(1, 2, 0)[:,:,1],masks[i,:,:,1]]
      plt.figure(figsize = (10, 10))
      for j in range(len(imgs)):
        plt.subplot(1, len(imgs), j+1)
        plt.imshow(imgs[j])
      plt.show()
  
  else:
    res = make_pred(outputs.permute(0,2,3,1))
    for i in range(BATCH_SIZE):
      imgs = [inputs[i].permute(1, 2, 0)[:,:,0], outputs[i].permute(1, 2, 0)[:,:,0],outputs[i].permute(1, 2, 0)[:,:,1],outputs[i].permute(1, 2, 0)[:,:,2],res[i,:,:,0], masks[i]]
      plt.figure(figsize = (10, 10))
      for j in range(len(imgs)):
        plt.subplot(1, len(imgs), j+1)
        plt.imshow(imgs[j])
      plt.show()

def make_pred(output):
  x = np.zeros((output.shape[0], output.shape[1],output.shape[2], 1))

  m_all = (output[:,:,:,0] == 0)
  m_or = (output[:,:,:,1] == 1)
  m_tu = (output[:,:,:,2] == 1)
  m_organ = np.logical_and(m_all, m_or)
  m_tu_2 = np.logical_and(m_all, (m_or == 0))

  #x[:,:,:,0][m_all == 1] = 1
  #x[:,:,:,0][m_or == 1] = 1
  x[:,:,:,0][m_organ == 1] = 1
  x[:,:,:,0][m_tu == 1] = 2
  x[:,:,:,0][m_tu_2 == 1] = 2

  return x

def iou_score(output, target):
    smooth = 1e-6

    if torch.is_tensor(output):
        output = output.data.detach().cpu().permute(0, 2, 3, 1).numpy()
    
    if torch.is_tensor(target):
        target = target.data.detach().cpu().numpy()

    output_ = (output>0.5)
    target_ = (target>0.5)

    intersection = (output_ & target_).sum()
    union = (output_ | target_).sum()

    return (intersection + smooth) / (union + smooth)

def iouScore(output, target):
  if torch.is_tensor(output):
    output = output.data.detach().cpu().permute(0, 2, 3, 1)
    if not SIGMOID:
      output = torch.sigmoid(output)
    output = make_pred(output>0.5)
    
  if torch.is_tensor(target):
    target = target.data.detach().cpu().numpy()
  
  target = np.argmax(target, axis = -1)

  jac = 0.0
  for i in range(4):
    out, tar = output[i], target[i]
    out = np.reshape(out, 512*512)
    tar = np.reshape(tar, 512*512)
    jac += jaccard_similarity_score(out, tar, normalize = True)

  return jac / 4.0

  return jac

def dice_coef(output, target):
    smooth = 1e-6

    output = output.view(-1).data.cpu().numpy()
    target = target.view(-1).data.cpu().numpy()
    intersection = (output * target).sum()

    return (2. * intersection + smooth) / (output.sum() + target.sum() + smooth)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BCEDiceLoss(nn.Module):
    def __init__(self, t_normedWeights, v_normedWeights):
        super().__init__()
        self.t_weight = t_normedWeights
        self.v_weight = v_normedWeights

    def forward(self, inputs, targets, mode):
        if mode == 'train':
          if not SIGMOID:
            bce = F.binary_cross_entropy_with_logits(inputs, targets, weight = self.t_weight, reduction = 'mean')
            smooth = 1e-6

            inputs = torch.sigmoid(inputs)
          else:
            bce = F.binary_cross_entropy(inputs, targets, weight = self.t_weight, reduction = 'mean')
            smooth = 1e-6

          dims = (1, 2)
          for i in range(3):
            input, target = inputs[:, i, :, :], targets[:, i, :, :]
            intersection = torch.sum(input * target, dims) * self.t_weight[i]
            cardinality = torch.sum(input + target, dims) * self.t_weight[i]

        else:
          if not SIGMOID:
            bce = F.binary_cross_entropy_with_logits(inputs, targets, weight = self.v_weight, reduction = 'mean')
            smooth = 1e-6

            inputs = torch.sigmoid(inputs)
          else:
            bce = F.binary_cross_entropy(inputs, targets, weight = self.v_weight, reduction = 'mean')
            smooth = 1e-6

          dims = (1, 2)
          for i in range(3):
            input, target = inputs[:, i, :, :], targets[:, i, :, :]
            intersection = torch.sum(input * target, dims) * self.v_weight[i]
            cardinality = torch.sum(input + target, dims) * self.v_weight[i]

        

        dice_score = 2. * (smooth+intersection) / (cardinality + smooth)
        dice_score = torch.mean(1. - dice_score)

        return 0.5 * (bce + dice_score)

In [ ]:
import cv2
from glob import glob
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
from pydicom import dcmread
import matplotlib.image as mpimg
#from utils import get_mask

class MOAIDataset(Dataset):
    def __init__(self, dir, transform = None):
        self.mask_dir = dir
        self.file_no = list(map(lambda x: x.split('/')[-2] +'/'+ (x.split('/')[-1].split('.')[0] + '.dcm'),self.mask_dir))
        self.image_dir = list(map(lambda x: TRAIN_IMAGE_DIR + '/'+ x, self.file_no))
        self.transform = transform

    
    def __len__(self):
        return len(self.image_dir)
    
    def __getitem__(self, index):
        img_path = self.image_dir[index]  # .dcm으로 저장된 DCM format의 CT 데이터
        mask_path = self.mask_dir[index]  # .png로 저장된 PNG format의 ground truth data
        

        image = read_dicom(img_path, 100, 50, 1)
        mask = mpimg.imread(mask_path)
        mask = np.around(mask, 8)

        m_bg = get_mask(mask)
        m_tu = get_mask(mask, 'tumor')
        m_or = get_mask(mask, 'organ')

        m_bg = np.expand_dims(m_bg, axis = -1)
        m_tu = np.expand_dims(m_tu, axis = -1)
        m_or = np.expand_dims(m_or, axis = -1)
        m_fn = np.concatenate([m_bg, m_or, m_tu], axis = -1)
        #m_fn = np.concatenate([m_or, m_tu], axis = -1)
        #m_fn = np.zeros((512, 512, 1))
        #m_fn[m_tu == 1] = 2
        #m_fn[m_or == 1] = 1

        
        # mask = np.array(cv2.imread(mask_path)[:,:,0]) # shape = (512, 512)

        if self.transform is not None:
            augmentations = self.transform(image = image, mask = m_fn)
            image = augmentations['image']
            m_fn = augmentations['mask']

        return image, m_fn.float()

In [ ]:
import torch
import torch.nn as nn
from albumentations.pytorch import ToTensorV2

class VGGBlock(nn.Module):
    def __init__(self, ch_in, ch_mid, ch_out):
        super().__init__()
        self.relu = nn.ReLU(inplace = True)
        self.conv1 = nn.Conv2d(ch_in, ch_mid, kernel_size = 3, padding = 1)
        self.bn1 = nn.BatchNorm2d(ch_mid)
        self.conv2 = nn.Conv2d(ch_mid, ch_out, kernel_size = 3, padding = 1)
        self.bn2 = nn.BatchNorm2d(ch_out)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        return out

class UNetPP(nn.Module):
    def __init__(self,ch_in,ch_out, deep_supervision = True, sigmoid = SIGMOID):
        super().__init__()

        #nb_filter = [16, 32, 64, 128, 256]
        nb_filter = [32, 64, 128, 256, 512]

        self.deep_supervision = deep_supervision
        self.sigmoid = sigmoid
        self.pool = nn.MaxPool2d(2, 2)
        self.up = nn.Upsample(scale_factor = 2, mode = 'bilinear', align_corners = True)

        self.conv0_0 = VGGBlock(ch_in, nb_filter[0], nb_filter[0])
        self.conv1_0 = VGGBlock(nb_filter[0], nb_filter[1], nb_filter[1])
        self.conv2_0 = VGGBlock(nb_filter[1], nb_filter[2], nb_filter[2])
        self.conv3_0 = VGGBlock(nb_filter[2], nb_filter[3], nb_filter[3])
        self.conv4_0 = VGGBlock(nb_filter[3], nb_filter[4], nb_filter[4])

        self.conv0_1 = VGGBlock(nb_filter[0]+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_1 = VGGBlock(nb_filter[1]+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_1 = VGGBlock(nb_filter[2]+nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv3_1 = VGGBlock(nb_filter[3]+nb_filter[4], nb_filter[3], nb_filter[3])

        self.conv0_2 = VGGBlock(nb_filter[0]*2+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_2 = VGGBlock(nb_filter[1]*2+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_2 = VGGBlock(nb_filter[2]*2+nb_filter[3], nb_filter[2], nb_filter[2])

        self.conv0_3 = VGGBlock(nb_filter[0]*3+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_3 = VGGBlock(nb_filter[1]*3+nb_filter[2], nb_filter[1], nb_filter[1])

        self.conv0_4 = VGGBlock(nb_filter[0]*4+nb_filter[1], nb_filter[0], nb_filter[0])

        if self.deep_supervision:
          if self.sigmoid:
            self.final1 = nn.Sequential(nn.Conv2d(nb_filter[0], ch_out, kernel_size=1), nn.Sigmoid())
            self.final2 = nn.Sequential(nn.Conv2d(nb_filter[0], ch_out, kernel_size=1), nn.Sigmoid())
            self.final3 = nn.Sequential(nn.Conv2d(nb_filter[0], ch_out, kernel_size=1), nn.Sigmoid())
            self.final4 = nn.Sequential(nn.Conv2d(nb_filter[0], ch_out, kernel_size=1), nn.Sigmoid())
          else:
            self.final1 = nn.Conv2d(nb_filter[0], ch_out, kernel_size=1)
            self.final2 = nn.Conv2d(nb_filter[0], ch_out, kernel_size=1)
            self.final3 = nn.Conv2d(nb_filter[0], ch_out, kernel_size=1)
            self.final4 = nn.Conv2d(nb_filter[0], ch_out, kernel_size=1)

        else:
            self.final = nn.Conv2d(nb_filter[0], ch_out, kernel_size=1)
    
    def forward(self, input):
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1))

        if self.deep_supervision:
            output1 = self.final1(x0_1)
            output2 = self.final2(x0_2)
            output3 = self.final3(x0_3)
            output4 = self.final4(x0_4)
            return [output1, output2, output3, output4]

        else:
            output = self.final(x0_4)
            return output



- DiceLosswith Logits는 점점 감소하여야 하고, IOUScore의 경우에는 계속해서 증가하여야 한다.


In [ ]:
def calc_weights(mask_dir):
  nSamples = [0,0,0]
  for idx, dir in enumerate(mask_dir):
    mask = mpimg.imread(dir)
    mask = np.round(mask, 8)
    if get_mask_bool(mask, 'tumor'):
      nSamples[2] += 1
    if get_mask_bool(mask, 'organ'):
      nSamples[1] += 1
    nSamples[0] += 1
  normedWeights = [1-(x/sum(nSamples)) for x in nSamples]
  normedWeights = torch.FloatTensor(normedWeights).to(DEVICE)

  return normedWeights
    

In [ ]:
import os
from collections import OrderedDict
from glob import glob

import pandas as pd
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from albumentations.augmentations import transforms
from albumentations.core.composition import Compose, OneOf
from sklearn.model_selection import train_test_split
from torch.optim import lr_scheduler
from tqdm import tqdm

def train(train_loader, model, criterion, optimizer, deep_supervision = True):
    avg_meters = {'loss': AverageMeter(),
                  'iou': AverageMeter()}

    model.train()
    loop = tqdm(train_loader)

    for idx, (image, target) in enumerate(loop):
        image = image.to(DEVICE)
        target = target.to(DEVICE)

        # compute output
        if deep_supervision:
            outputs = model(image)
            loss = 0
            for output in outputs:
                loss += criterion(output.permute(0,2, 3, 1).float(), target, mode = 'train')
            loss /= len(outputs)   # BATCHSIZE
            iou = iouScore(outputs[-1], target)
            if idx % 50 ==0:
              visualize(image, outputs[-1], target, 3)
        else:
            output = model(image)
            loss = criterion(output, target)
            iou = iou_score(output, target)

        # compute gradient and do optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_meters['loss'].update(loss.item(), image.shape[0])
        avg_meters['iou'].update(iou, image.shape[0])

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('iou', avg_meters['iou'].avg),
        ])
        loop.set_postfix(postfix)
        #pbar.set_postfix(postfix)
        #pbar.update(1)
    #pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('iou', avg_meters['iou'].avg)])
    
def validate(valid_loader, model, criterion, scheduler, deep_supervision = True):
    avg_meters = {'loss': AverageMeter(),
                  'iou': AverageMeter()}

    # switch to evaluate mode
    model.eval()
    loop = tqdm(valid_loader)

    with torch.no_grad():
        for iter, (image, target) in enumerate(loop):
            image = image.to(DEVICE)
            target = target.to(DEVICE)

            # compute output
            if deep_supervision:
                outputs = model(image)
                
                loss = 0
                out_fin = outputs[-1]
                for output in outputs:
                  loss += criterion(output.permute(0, 2, 3, 1).float(), target, mode = 'valid')
                loss /= len(outputs)
                iou = iouScore(outputs[-1], target)
                if iter %50 ==0:
                    visualize(image, out_fin, target, 3)
            else:
                output = model(image)
                loss = dice_coef(output.permute(0,2,3,1).float(), target)
                iou = iou_score(output, target)

            avg_meters['loss'].update(loss.item(), image.shape[0])
            avg_meters['iou'].update(iou, image.shape[0])

            postfix = OrderedDict([
                ('loss', avg_meters['loss'].avg),
                ('iou', avg_meters['iou'].avg),
            ])
            loop.set_postfix(postfix)
            #pbar.set_postfix(postfix)
            #pbar.update(1)
        #pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('iou', avg_meters['iou'].avg)])

def main():
    model = UNetPP(ch_in = 1, ch_out = 3, sigmoid = SIGMOID).to(DEVICE)
    
    optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience = 2)

    

    
    if LOAD_MODEL:
      load_checkpoint(torch.load(CHECKPOINT_DIR), model)

    
    for epoch in range(NUM_EPOCHS):

      dcom_dirs = np.array(sorted(glob(TRAIN_IMAGE_DIR + '/*/*')))
      mask_dirs = np.array(sorted(glob(TRAIN_MASK_DIR + '/*/*')))

      mask_train_dirs, mask_valid_dirs = train_test_split(mask_dirs, test_size = 0.2, shuffle = True)

      t_normedWeights, v_normedWeights = calc_weights(mask_train_dirs), calc_weights(mask_valid_dirs)

     
      #criterion = nn.BCELoss(weight=normedWeights).to(DEVICE)
      criterion = BCEDiceLoss(t_normedWeights, v_normedWeights).to(DEVICE)

  

      train_transform = Compose([
        transforms.RandomRotate90(always_apply = True),
        transforms.Flip(),
        transforms.Resize(512, 512),
        transforms.Normalize(mean = 0.0, std = 1.0, max_pixel_value = 1.0),
        ToTensorV2()])
    
      val_transform = Compose([
        transforms.Resize(512, 512),
        transforms.Normalize(mean = 0.0, std = 1.0, max_pixel_value = 1.0),
        ToTensorV2()
        ])

      train_loader, valid_loader = get_loaders(
        mask_train_dirs, mask_valid_dirs, BATCH_SIZE,
        train_transform, val_transform, num_workers = NUM_WORKERS, pin_memory = PIN_MEMORY
        )


      best_iou, trigger = 0, 0

      print("=>Training for epoch %d" %epoch)

      train_log = train(train_loader, model, criterion, optimizer)
      valid_log = validate(valid_loader, model, criterion, scheduler)

      scheduler.step(valid_log['loss'])

      trigger += 1

      if valid_log['iou'] > best_iou:
          checkpoint = {
              "state_dict" : model.state_dict(),
              "optimizer" : optimizer.state_dict()
                }
          save_checkpoint(checkpoint)
          best_iou = valid_log['iou']
          print("=> SAVED BEST IOU")
          trigger = 0
        
      torch.cuda.empty_cache()


if __name__ == "__main__":
    main()





In [ ]:
model = UNetPP(ch_in = 1, ch_out = 3, sigmoid = SIGMOID).to(DEVICE)
load_checkpoint(torch.load(CHECKPOINT_DIR), model)

test_dir = '/content/drive/Shareddrives/Kaggle_MOAI2021/data/test/DICOM'
test_images = np.array(sorted(glob(test_dir + '/*/*')))

import os
from PIL import Image
import argparse
import numpy as np

def rle_encode(mask_image):
    pixels = mask_image.flatten()
    # We avoid issues with '1' at the start or end (at the corners of
    # the original image) by setting those pixels to '0' explicitly.
    # We do not expect these to be non-zero for an accurate mask,
    # so this should not harm the score.
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs


def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def submit_encode(mask):
  x = np.zeros((mask.shape[0], mask.shape[1], 1))
  
  mask = (mask > 0.5).float()
  # 전체 organ + tumor모든 부분을 예측
  m_all = (mask[:,:,0] == 0.0)
  m_tu = mask[:,:,2]
  m_or = mask[:,:,1]
  m_organ = np.logical_and(m_all, m_or)
  m_tu_2 = np.logical_and(m_all, (m_or == 0.0))
  
  #x[:,:,0][m_all == 1] = 1
  x[:,:,0][m_organ == 1] = 1
  
  # 모델의 organ예측과 all 부분 예측의 공통 부분을 organ으로 생각

  x[:,:,0][m_tu == 1] = 2
  x[:,:,0][m_tu_2 == 1] = 2
  
  return x

=>loading checkpoint...


In [ ]:
# submission을 위한 prediction값을 저장해 준다.
# 출력값에 대해서 sigmoid 를 이용해서 0과 1사이의 값으로 바꾸어 주고
# my_encode()함수를 사용해서 
# channel별로 0, 1, 2의 값으로 저장해 준다.
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import os



import os
from glob import glob
import numpy as np
from tqdm import tqdm
import torch

test_dir = '/content/drive/Shareddrives/Kaggle_MOAI2021/data/test/DICOM'
test_ids = os.listdir(test_dir)
test_images = np.array(sorted(glob(test_dir + '/*/*')))

preds_test = []
test_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(
      mean = 0.0,
      std = 1.0,
      max_pixel_value = 1.0
      ),
    ToTensorV2()
])

for n, path in tqdm(enumerate(test_images), total=len(test_images)):
    id = path.split('/')[-2]
    img = read_dicom(path, 100, 50,1)
    inp = test_transform(image = img)['image']
    inp = torch.from_numpy(np.expand_dims(inp, axis = 0)).to(DEVICE)


    out_first = model(inp)[-1]
    if not SIGMOID:
      out_first = torch.sigmoid(out_first)
    out = out_first[0].permute(1, 2, 0).float()
    #print(np.unique(out[:,:,0]),np.unique(out[:,:,1]),np.unique(out[:,:,2]))
    #out = (out > 0.5).float()
    


    out_res = submit_encode(out.detach().cpu())
    preds_test.append(out_res)
    plt.figure(figsize = (6,6))
    plt.subplot(1, 3, 1)
    plt.imshow(img[:,:,0])
    plt.subplot(1, 3, 2)
    plt.imshow(out.detach().cpu())
    plt.subplot(1, 3, 3)
    plt.imshow(out_res[:,:,0]/2)
    plt.show()
    print(np.unique(out_res), (out_res == 2).sum())
   

In [ ]:
import pandas as pd
# rlencoding방법을 적용해서 csv파일로 만들어 제출이 가능하도록 한다..
preds_string = []
for i in tqdm(range(0, len(preds_test), 64)):
    sample = preds_test[i:i+64].copy()
    for label_code in [1,2]:
        tmp=[]
        for s in sample:
            s = np.equal(s, label_code).flatten()*1
            tmp+=s.tolist()
        enc = rle_to_string(rle_encode(np.array(tmp)))

        preds_string.append(enc)


100%|██████████| 83/83 [04:21<00:00,  3.15s/it]


In [ ]:
sample_submission = pd.read_csv('/content/drive/Shareddrives/Kaggle_MOAI2021/data/sample_submission.csv')
sample_submission['EncodedPixels'] = preds_string
sample_submission.to_csv('/content/drive/Shareddrives/Kaggle_MOAI2021/checkpoints/submission.csv', index=False)